In [0]:
import numpy as np
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


## Obtain the train and test data

In [3]:
train = pd.read_csv('/content/gdrive/My Drive/Journal Paper/Human Activity Detection/UCI HAR Dataset/csv_files/train.csv')
test = pd.read_csv('/content/gdrive/My Drive/Journal Paper/Human Activity Detection/UCI HAR Dataset/csv_files/test.csv')
print(train.shape, test.shape)



(7352, 564) (2947, 564)


In [4]:
train.head(3)

,1tBodyAccmeanX,2tBodyAccmeanY,3tBodyAccmeanZ,4tBodyAccstdX,5tBodyAccstdY,6tBodyAccstdZ,7tBodyAccmadX,8tBodyAccmadY,9tBodyAccmadZ,10tBodyAccmaxX,11tBodyAccmaxY,12tBodyAccmaxZ,13tBodyAccminX,14tBodyAccminY,15tBodyAccminZ,16tBodyAccsma,17tBodyAccenergyX,18tBodyAccenergyY,19tBodyAccenergyZ,20tBodyAcciqrX,21tBodyAcciqrY,22tBodyAcciqrZ,23tBodyAccentropyX,24tBodyAccentropyY,25tBodyAccentropyZ,26tBodyAccarCoeffX1,27tBodyAccarCoeffX2,28tBodyAccarCoeffX3,29tBodyAccarCoeffX4,30tBodyAccarCoeffY1,31tBodyAccarCoeffY2,32tBodyAccarCoeffY3,33tBodyAccarCoeffY4,34tBodyAccarCoeffZ1,35tBodyAccarCoeffZ2,36tBodyAccarCoeffZ3,37tBodyAccarCoeffZ4,38tBodyAcccorrelationXY,39tBodyAcccorrelationXZ,40tBodyAcccorrelationYZ,...,525fBodyBodyAccJerkMagmaxInds,526fBodyBodyAccJerkMagmeanFreq,527fBodyBodyAccJerkMagskewness,528fBodyBodyAccJerkMagkurtosis,529fBodyBodyGyroMagmean,530fBodyBodyGyroMagstd,531fBodyBodyGyroMagmad,532fBodyBodyGyroMagmax,533fBodyBodyGyroMagmin,534fBodyBodyGyroMagsma,535fBodyBodyGyroMagenergy,536fBodyBodyGyroMagiqr,537fBodyBodyGyroMagentropy,538fBodyBodyGyroMagmaxInds,539fBodyBodyGyroMagmeanFreq,540fBodyBodyGyroMagskewness,541fBodyBodyGyroMagkurtosis,542fBodyBodyGyroJerkMagmean,543fBodyBodyGyroJerkMagstd,544fBodyBodyGyroJerkMagmad,545fBodyBodyGyroJerkMagmax,546fBodyBodyGyroJerkMagmin,547fBodyBodyGyroJerkMagsma,548fBodyBodyGyroJerkMagenergy,549fBodyBodyGyroJerkMagiqr,550fBodyBodyGyroJerkMagentropy,551fBodyBodyGyroJerkMagmaxInds,552fBodyBodyGyroJerkMagmeanFreq,553fBodyBodyGyroJerkMagskewness,554fBodyBodyGyroJerkMagkurtosis,555angletBodyAccMeangravity,556angletBodyAccJerkMeangravityMean,557angletBodyGyroMeangravityMean,558angletBodyGyroJerkMeangravityMean,559angleXgravityMean,560angleYgravityMean,561angleZgravityMean,subject,Activity,ActivityName
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,-0.567378,-0.744413,0.852947,0.685845,0.814263,-0.965523,-0.999945,-0.999863,-0.994612,-0.994231,-0.987614,-0.943220,-0.407747,-0.679338,-0.602122,0.929294,-0.853011,0.359910,-0.058526,0.256892,-0.224848,0.264106,-0.095246,0.278851,-0.465085,0.491936,-0.190884,0.376314,0.435129,0.660790,...,-0.936508,0.346989,-0.516080,-0.802760,-0.980135,-0.961309,-0.973653,-0.952264,-0.989498,-0.980135,-0.999240,-0.992656,-0.701291,-1.000000,-0.128989,0.586156,0.374605,-0.991990,-0.990697,-0.989941,-0.992448,-0.991048,-0.991990,-0.999937,-0.990458,-0.871306,-1.000000,-0.074323,-0.298676,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627,1,5,STANDING
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,-0.557851,-0.818409,0.849308,0.685845,0.822637,-0.981930,-0.999991,-0.999788,-0.998405,-0.999150,-0.977866,-0.948225,-0.714892,-0.500930,-0.570979,0.611627,-0.329549,0.284213,0.284595,0.115705,-0.090963,0.294310,-0.281211,0.085988,-0.022153,-0.016657,-0.220643,-0.013429,-0.072692,0.579382,...,-0.841270,0.532061,-0.624871,-0.900160,-0.988296,-0.983322,-0.982659,-0.986321,-0.991829,-0.988296,-0.999811,-0.993979,-0.720683,-0.948718,-0.271958,-0.336310,-0.720015,-0.995854,-0.996399,-0.995442,-0.996866,-0.994440,-0.995854,-0.999981,-0.994544,-1.000000,-1.000000,0.158075,-0.595051,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317,1,5,STANDING
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,-0.557851,-0.818409,0.843609,0.682401,0.839344,-0.983478,-0.999969,-0.999660,-0.999470,-0.997130,-0.964810,-0.974675,-0.592235,-0.485821,-0.570979,0.273025,-0.086309,0.337202,-0.164739,0.017150,-0.074507,0.342256,-0.332564,0.239281,-0.136204,0.173863,-0.299493,-0.124698,-0.181105,0.608900,...,-0.904762,0.660795,-0.724697,-0.928539,-0.989255,-0.986028,-0.984274,-0.990979,-0.995703,-0.989255,-0.999854,-0.993238,-0.736521,-0.794872,-0.212728,-0.535352,-0.871914,-0.995031,-0.995127,-0.994640,-0.996060,-0.995866,-0.995031,-0.999973,-0.993755,-1.000000,-0.555556,0.414503,-0.390748,-0.760104,-0.

In [5]:
# get X_train and y_train from csv files
X_train = train.drop(['subject', 'Activity', 'ActivityName'], axis=1)
y_train = train.Activity
print(y_train)

0       5
1       5
2       5
3       5
4       5
       ..
7347    2
7348    2
7349    2
7350    2
7351    2
Name: Activity, Length: 7352, dtype: int64


In [6]:
# get X_test and y_test from test csv file
X_test = test.drop(['subject', 'Activity', 'ActivityName'], axis=1)
y_test = test.Activity
print(y_test)

0       5
1       5
2       5
3       5
4       5
       ..
2942    2
2943    2
2944    2
2945    2
2946    2
Name: Activity, Length: 2947, dtype: int64


In [7]:
print('X_train and y_train : ({},{})'.format(X_train.shape, y_train.shape))
print('X_test  and y_test  : ({},{})'.format(X_test.shape, y_test.shape))

X_train and y_train : ((7352, 561),(7352,))
X_test  and y_test  : ((2947, 561),(2947,))


# Implementing Simple 2 Layer ANN

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


In [0]:
# Perform one hot encoding on Y  or Dependent Varibles
from keras import utils as np_utils
y_train_en = np_utils.to_categorical(y_train)
y_train_en = y_train_en[:,1:7]

In [0]:
# Perform one hot encoding on Y  or Dependent Varibles
y_test_en = np_utils.to_categorical(y_test)
y_test_en = y_test_en[:,1:7]

In [11]:
print(X_train.shape)
print(y_train_en.shape)
print(X_test.shape)
print(y_test_en.shape)

(7352, 561)
(7352, 6)
(2947, 561)
(2947, 6)


In [12]:
# Converting the Pandas dataframe into numpy array
X_train_np = X_train.to_numpy()
X_test_np = X_test.to_numpy()

print(type(X_train_np))
print(type(y_train_en))
print(type(X_test_np))
print(type(y_test_en))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


# Keras ANN

In [13]:
# Initiliazing the sequential model
model = Sequential()
# Layer 1
model.add(Dense(128, input_dim=561, activation='relu', name='FC0'))
model.add(Dropout(0.5))

# Layer 2
model.add(Dense(6, activation='softmax', name='FC2'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
FC0 (Dense)                  (None, 128)               71936     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
FC2 (Dense)                  (None, 6)                 774       
Total params: 72,710
Trainable params: 72,710
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
# Training the model
model.fit(X_train_np,
          y_train_en,
          batch_size=64,
          validation_data=(X_test_np, y_test_en),
          epochs=100)

Train on 7352 samples, validate on 2947 samples
Epoch 1/100
7352/7352 [==============================] - 1s 94us/step - loss: 0.7204 - accuracy: 0.7082 - val_loss: 0.3449 - val_accuracy: 0.8948
Epoch 2/100
7352/7352 [==============================] - 0s 54us/step - loss: 0.3288 - accuracy: 0.8760 - val_loss: 0.2416 - val_accuracy: 0.9240
Epoch 3/100
7352/7352 [==============================] - 0s 57us/step - loss: 0.2462 - accuracy: 0.9083 - val_loss: 0.1974 - val_accuracy: 0.9376
Epoch 4/100
7352/7352 [==============================] - 0s 58us/step - loss: 0.2050 - accuracy: 0.9251 - val_loss: 0.1694 - val_accuracy: 0.9393
Epoch 5/100
7352/7352 [==============================] - 0s 57us/step - loss: 0.1753 - accuracy: 0.9350 - val_loss: 0.1718 - val_accuracy: 0.9308
Epoch 6/100
7352/7352 [==============================] - 0s 56us/step - loss: 0.1531 - accuracy: 0.9434 - val_loss: 0.1658 - val_accuracy: 0.9352
Epoch 7/100
7352/7352 [==============================] - 0s 57us/step - loss

In [0]:
# Function for calculating the Confusion Matrix
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

In [18]:
# Confusion Matrix
confusion_matrix(y_test_en, model.predict(X_test_np))

Pred,LAYING,SITTING,STANDING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,,,,
LAYING,537,0,0,0,0,0
SITTING,0,452,37,0,0,2
STANDING,0,40,491,1,0,0
WALKING,0,0,0,494,2,0
WALKING_DOWNSTAIRS,0,0,0,11,395,14
WALKING_UPSTAIRS,0,0,0,37,3,431


In [20]:
score = model.evaluate(X_test_np, y_test_en)
print(score)

2947/2947 [==============================] - 0s 26us/step
[0.17629853113047775, 0.9501187801361084]
